In [34]:
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.losses import CategoricalCrossentropy
from keras.metrics import CategoricalAccuracy, AUC
from keras.layers import Input, Dense, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [35]:
# Multi-class classification
BATCH_SIZE = 32
CLASS_MODE = "categorical"
COLOR_MODE = "grayscale"
DIRECTORY = "Covid19-dataset/train"

# Load image data
data_generator = ImageDataGenerator(
    rescale=1./255, 
    zoom_range=0.1,
    rotation_range=25, 
    width_shift_range=0.05, 
    height_shift_range=0.05)

# Create iterable objects from data_generator
train_iterator = data_generator.flow_from_directory(
    DIRECTORY,
    class_mode=CLASS_MODE,
    color_mode=COLOR_MODE,
    batch_size=BATCH_SIZE)

validation_iterator = data_generator.flow_from_directory(
    DIRECTORY,
    class_mode=CLASS_MODE,
    color_mode=COLOR_MODE,
    batch_size=BATCH_SIZE)

Found 251 images belonging to 3 classes.


Found 251 images belonging to 3 classes.


In [36]:
# Create model
model = Sequential()
model.add(Input(shape=(256, 256, 1)))
model.add(Conv2D(5, 5, strides=2, activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(3, 3, strides=1, activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dense(64, activation="relu"))
model.add(Flatten())
model.add(Dense(3, activation="softmax"))

In [37]:
# Compile and train model
model.compile(loss=CategoricalCrossentropy(), 
              optimizer=Adam(learning_rate=0.001), 
              metrics=[CategoricalAccuracy(), AUC()])
history = model.fit(train_iterator, 
          steps_per_epoch=train_iterator.samples/BATCH_SIZE,
          epochs=10,
          validation_data=validation_iterator,
          validation_steps=validation_iterator.samples / BATCH_SIZE)

# Evaluate model
print(model.evaluate(validation_iterator))

Epoch 1/10
7/7 [==============================] - 2s 232ms/step - loss: 1.0862 - categorical_accuracy: 0.4821 - auc_5: 0.6517 - val_loss: 1.0507 - val_categorical_accuracy: 0.4701 - val_auc_5: 0.7287
Epoch 2/10
7/7 [==============================] - 2s 206ms/step - loss: 0.9811 - categorical_accuracy: 0.5219 - auc_5: 0.7739 - val_loss: 0.8669 - val_categorical_accuracy: 0.7530 - val_auc_5: 0.8973
Epoch 3/10
7/7 [==============================] - 2s 204ms/step - loss: 0.7344 - categorical_accuracy: 0.7649 - auc_5: 0.9208 - val_loss: 0.6051 - val_categorical_accuracy: 0.7371 - val_auc_5: 0.9270
Epoch 4/10
7/7 [==============================] - 2s 204ms/step - loss: 0.4932 - categorical_accuracy: 0.8566 - auc_5: 0.9618 - val_loss: 0.4071 - val_categorical_accuracy: 0.8645 - val_auc_5: 0.9713
Epoch 5/10
7/7 [==============================] - 2s 205ms/step - loss: 0.3896 - categorical_accuracy: 0.8446 - auc_5: 0.9630 - val_loss: 0.3251 - val_categorical_accuracy: 0.8526 - val_auc_5: 0.9747


In [38]:
# Print classification report
test_steps_per_epoch = np.math.ceil(validation_iterator.samples / validation_iterator.batch_size)
predictions = model.predict(validation_iterator, steps=test_steps_per_epoch)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = validation_iterator.classes
class_labels = list(validation_iterator.class_indices.keys())
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report) 

8/8 [==============================] - 1s 84ms/step
              precision    recall  f1-score   support

       Covid       0.41      0.40      0.40       111
      Normal       0.25      0.33      0.29        70
   Pneumonia       0.28      0.21      0.24        70

    accuracy                           0.33       251
   macro avg       0.32      0.31      0.31       251
weighted avg       0.33      0.33      0.33       251



In [39]:
confusion_matrix = confusion_matrix(true_classes, predicted_classes)
print(confusion_matrix)

[[44 43 24]
 [33 23 14]
 [30 25 15]]


In [40]:
# Save model
model.save("cnn_model")

INFO:tensorflow:Assets written to: cnn_model\assets


INFO:tensorflow:Assets written to: cnn_model\assets
